In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import walnut

# Example 4.1

### Convolutional Neural Network: MNIST

The goal of this model is to classify images of hand-written digits.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/competitions/digit-recognizer/data and place it into the *data* directory. Only using the official training data for training, validation and testing, since it is just to showcase the framework.

In [ ]:
data = pd.read_csv('data/mnist/train.csv')
data.head()

In [ ]:
tensor = walnut.pd_to_tensor(data)
train, val, test = walnut.preprocessing.split_train_val_test(tensor, ratio_val=0.01, ratio_test=0.02)

In [ ]:
x_train, y_train = train[:, 1:], train[:, 0].astype("int")
x_val, y_val = val[:, 1:], val[:, 0].astype("int")
x_test, y_test = test[:, 1:], test[:, 0].astype("int")

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 1 , 28, -1))
x_val = x_val.reshape((x_val.shape[0], 1, 28, -1))
x_test = x_test.reshape((x_test.shape[0], 1, 28, -1))

print (f'{x_train.shape=}')
print (f'{y_train.shape=}')

print (f'{x_val.shape=}')
print (f'{y_val.shape=}')

print (f'{x_test.shape=}')
print (f'{y_test.shape=}')

Normalization

In [ ]:
x_train = x_train.astype("float32") / 255.0
x_val = x_val.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

### Step 2: Build the neural network structure

In [ ]:
import walnut.nn as nn
from walnut.nn.layers import *

model = nn.Sequential([
    Convolution2d(1, 32, kernel_size=(3, 3), use_bias=False), Batchnorm(32), Relu(), # bias not needed when using norm
    MaxPooling2d(kernel_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Linear(13*13*32, 200, use_bias=False), Batchnorm(200), Relu(),
    Linear(200, 10)
])

In [ ]:
model.compile(
    optimizer=nn.optimizers.Adam(l_r=1e-3),
    loss_fn=nn.losses.Crossentropy(),
    metric=nn.metrics.Accuracy()
)

In [ ]:
from walnut.nn.analysis import model_summary
model_summary(model, (1, 28, 28))

### Step 3: Train the model

In [ ]:
epochs = 100
batch_size = 32

train_loss_hist, val_loss_hist = model.train(x_train, y_train, epochs=epochs, batch_size=batch_size, val_data=(x_val, y_val))

In [ ]:
n = epochs // 10  # average over the last n values to reduce noise caused by a small batch size

traces = {
    "train_loss" : [np.average(train_loss_hist[i-min(n-1, i):i+1]) for i in range(len(train_loss_hist))],
    "val_loss" : val_loss_hist
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="loss history", x_label="epoch", y_label="loss")

### Step 4: Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'loss {loss:.4f}')
print(f'accuracy {accuracy*100:.2f}')

In [ ]:
predictions = model(x_test)
nn.analysis.plot_confusion_matrix(predictions, y_test, figsize=(5,5))

### Step 5: Explore the inner workings
Pick a random image from the testing dataset.

In [ ]:
i = random.randint(0, x_test.len)
image = np.moveaxis(x_test[i].data, 0, -1)
plot = plt.imshow(image, cmap='gray')
plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

Use it to predict a number and show the probability distribution of the outcome.

In [ ]:
image_tensor = walnut.expand_dims(x_test[i], 0)
print(f"correct label: {y_test[i].item()}")
logits = model(image_tensor)
print(f"predicted label: {logits.argmax(-1).item()}")
nn.analysis.plot_probabilities(logits, figsize=(6, 3))

Every layer of the model can be accessed to explore their output. Here we iterate over all the kernels of the convolutional layer to explore what they learned to focus on in images.

In [ ]:
channels = {f"{i + 1} {l.__class__.__name__}" : l.y.data[0].copy() for i, l in enumerate(model.layers) if l.__class__.__name__ == "Convolution2d"}
nn.analysis.plot_images(channels, figsize=(40, 40), cmap="gray")